# Creating NoSQL database with modules from STEPmod

For this we need to extract the data from all files in STEPmod and organize it as key-values tables to import in MongoDB.
First we create the data base.

In [2]:
from pymongo import MongoClient
client = MongoClient()
db = client.step

## Import data into python dictionaries

### Get the list of files to import

#### Set the working directory

In [3]:
import re
import os

#### Get modules, schema and entity names with their relations

Enter schemas and entities as dictionaries:
* example for modules and schemas: {*
* example for schemas and modules: {"schema_id" : "Activity_arm", "entity" : "activity" }

In [87]:
os.chdir('/Users/aminatambengue/Documents/workspace/stepmod/data/modules')
md_list=list()
for item in os.listdir():
    if not item.startswith('.'):
        md_list.append(item)
c=str()
for elt in md_list:
#for elt in list(['activity','activity_method','ap242_managed_model_based_3d_engineering']): #for tests on a subset of stepmod
    entity= 'n'
    md=dict()
    md["module"]=elt.lower()
    #look for all .exp files
    path="/Users/aminatambengue/Documents/workspace/stepmod/data/modules/%s" %elt
    os.chdir(path)
    f = os.listdir()
    for i in f: #select file in folder
        #if (".exp" in i and "concatenated" not in i and "patch" not in i): # check that it's an express
        if (".exp" in i and "concatenated" not in i and "patch" not in i and "prev.exp" not in i):
            j = 1 #line number
            comment='n'
            h=open(i)
            for line in iter(h): #read it line by line
                #To exclude comments
                if "(*" in line:
                    comment='y'
                if "*)" in line:
                    comment='n'
                if comment == 'n':
                    sch=dict()
                    while entity == 'n':
                        if ("SCHEMA " in line and "from" not in line):
                            a=re.search("SCHEMA ",line)
                            b=re.search(";",line)
                            if b is not None:
                                c=line[a.end():b.start()]
                                md["_id"]=c  
                                #db.modules.insert_one(md) #enter list of modules and associated schemas
                        elif ("ENTITY " in line and "SELECT" not in line and "END_ENTITY" not in line):
                            entity = 'y'
                            a=re.search("ENTITY ",line)
                            if ";" in line:
                                b=re.search(";",line)
                                if b is not None:
                                    sch["entity"]=line[a.end():b.start()].lower()
                            else:
                                sch["entity"]=line[a.end():-1].lower()
                            sch["schema_id"]=c
                            #db.modules.insert_one(sch) #enter list of entities and associated schemas
                    #print([j,line,i,elt])
                j=j+1
            h.close()     

## Test the database
List of schemas containing an entity

In [86]:
cursor = db.modules.find({'entity':'activity'})
for document in cursor:
    print(document.get("schema_id"))

Activity_arm
Ap203_configuration_controlled_3d_design_of_mechanical_parts_and_assemblies_arm_lf
Ap209_multidisciplinary_analysis_and_design_arm_LF
Ap210_assembly_functional_interface_requirements_arm_LF
Ap210_assembly_functional_requirements_arm_LF
Ap210_assembly_physical_design_arm_LF
Ap210_assembly_physical_interface_requirements_arm_LF
Ap210_assembly_physical_requirements_arm_LF
Ap210_assembly_requirement_allocation_arm_LF
Ap210_assembly_technology_constraints_arm_LF
Ap210_connection_zone_based_model_extraction_arm_LF
Ap210_datum_difference_based_model_definition_arm_LF
Ap210_device_functional_and_physical_characterization_arm_LF
Ap210_electronic_assembly_interconnect_and_packaging_design_arm_LF
Ap210_functional_decomposition_arm_LF
Ap210_functional_requirement_allocation_arm_LF
Ap210_functional_specification_arm_LF
Ap210_interconnect_design_arm_LF
Ap210_interconnect_design_for_microwave_arm_LF
Ap210_interconnect_functional_requirements_arm_LF
Ap210_interconnect_physical_requirement

## Import attributes